# Big Data y Machine Learning para clasificación de galaxias

- [Almacenamiento HIVE - HUE - IMPALA](#hue-impala)
    - [Dataset SDSS_PhotObj](#phot)
    - [Dataset SDSS_SpecObj](#spec)
    - [Modelo relacional](#relacional)
        - [HIVE: External Table](#hive-external)
        - [HIVE: Optimized Row Columnar](#hive-orc)
    - [Administración HUE](#hue)
        - [HUE Queries](#hue-queries)
        - [HUE: Optimized Row Columnar](#hue-orc)
        - [HUE: Filtrado de datos](#hue-filtrado)
    - [Analítica HIVE - SPARK](#hive-spark)


<div id='xx' />

<div id='hue-impala' />

## Almacenamiento HIVE - HUE - IMPALA

Para un buen manejo de los datos y su posterior análisis es fundamental que hayamos definido un “correcto” modelo de datos. En esta sección vamos a explicar lo que es el modelo de datos relacional, implementaremos el modelo usando una de las herramientas del ecosistema de HADOOP llamada HIVE y, finalmente, sobre el modelo ya definido ejecutaremos una serie de consultas y análisis sencillos sobre los datos usando herramientas como HUE e IMPALA.

En primer lugar comprobamos que los datos transferidos en la sección anterior se han copiado correctamente:

---

`
[container]$ hdfs dfs -cat /user/hive/galaxias/SDSS_PhotObj/SDSS_PhotObj.csv | wc -l
25101
[container]$ hdfs dfs -cat /user/hive/galaxias/SDSS_SpecObj/SDSS_SpecObj.csv | wc -l
25101
[container]$ hdfs dfs -cat /user/hive/galaxias/SDSS_PhotObj/SDSS_PhotObj.csv | head -n1
objid,fileid,ra,dec,u,g,r,i,z,field
[container]$ hdfs dfs -cat /user/hive/galaxias/SDSS_SpecObj/SDSS_SpecObj.csv | head -n1
objid,redshift,plate,mjd,fiberid,class
`

---

Para poder acceder a la información que contiene estos ficheros de texto es necesario definir su modelo de datos. En primer lugar notamos que los ficheros *SDSS_PhotObj.csv y SDSS_SpecObj.csv* contienen 25100 filas de observaciones. Además podemos ver las variables de nuestro datos en la cabecera del archivo .csv, los cuales pasamos a detallar a continuación:

<div id='phot' />

##### SDSS_PhotObj.csv

- **objid (int)** 
    - identificador único del objeto en el catálogo de SDSSDR8.
- **fileid (int)**
    - alias del parámetro “dr7objid”, el identificador que usamos en este proyecto para el fichero que contiene la imagen correspondiente del objeto.
- **ra, dec (float)**
    - coordenadas celestes del objeto ('right ascension' y 'declination', en grados).
- **u,g,r,i y z (float)**
    - brillo (magnitud) medido en los diferentes filtros de colores por cada objeto. Las letras son nombres estándares de filtros que dejan pasar solo luz en un determinado intervalo de longitudes de onda. Cuanto menor es este valor, más brillante es el objeto.
- **field (int)**
    - número que identifica el área de cielo en la que seencuentra el objeto.
    
Si quisiéramos obtener las coordenadas celestes para la última observación (fila) de nuestro dataset de manera rápida podríamos lanzar el siguiente comando sobre HDFS:

---

`
[container]$ hdfs dfs -cat /user/cloudera/galaxias/SDSS_PhotObj/SDSS_PhotObj.csv | tail -n1 | cut -f "3,4" -d ","
120.93984795,44.83061183
`

---

Nuestro segundo dataset contiene los siguientes tipos de datos:

<div id='spec' />

##### SDSS_SpecObj.csv

- **objid (int)**
    - identificador único de la galaxia (objeto) en el catálogo SDSS.
- **redshift (float)**
    - estimación de corrimiento al rojo (alias 'Z'). Se trata de un indicador de la distancia del objeto.
- **plate, fiberid (int)**
    - identificadores de la instrumentación del telescopio que se ha utilizado para hacer la medida que ha llevado a la estimación de redshift.
- **mjd (int)**
    - identificador de la fecha en la que se ha tomado la medida (modified Julian date).
- **class (string)**
    - nombre de la clase del objeto, según las características de la luz emitida en el espectro electromagnético observado por SDSS.

<div id='relacional' />

### Modelo relacional

En nuestro caso vamos a almacenar nuestros datos en una base de datos relacional. En este tipo de modelo la información está organizada en tablas. Estas tablas están compuestas por registros (cada fila de la tabla sería un registro) y columnas (también llamadas campos). El modelo de datos relacional es el más utilizado en la actualidad para modelar problemas reales y administrar datos dinámicamente.

HIVE es un sistema de almacenamiento de datos construido sobre HADOOP para proporcionar agrupación, consulta y análisis de datos masivos. HIVE define un tipo especial de tabla, llamada *tabla externa*, que relaciona los campos de la propia tabla con los campos de un fichero. Por otro lado, IMPALA es un motor de consultas SQL open source para el procesamiento masivo de datos en paralelo.

Primero que nada veamos como interaccionar con HIVE y crear una base de datos:

---

`
[container]$ sudo su
[container]$ hive
[hive]$ show databases;
OK
default
Time taken: 70.633 seconds, Fetched: 1 row(s)
[hive]$ create database galaxias;
OK
Time taken: 0.086 seconds
[hive]$ show databases;
OK
default
galaxias
Time taken: 0.012 seconds, Fetched: 2 row(s)
[hive]$ use galaxias;
OK
Time taken: 0.021 seconds
`

---

<div id='hive-external' />

#### HIVE: External Table

Fijándonos en la sección anterior donde se definen los nombres de los campos y sus tipos, definimos la tabla externa correspondiente al fichero *SDSS_PhotObj.csv*:

---

~~~
CREATE EXTERNAL TABLE sdss_photobj_csv (
    objid BIGINT COMMENT 'identificador unico del objeto en el catalogo deSDSS',
    fileid BIGINT COMMENT 'alias del parametro \"dr7objid\", identificador que usamos en este proyecto para el fichero que contiene la imagen correspondiente del objeto',
    ra FLOAT COMMENT 'ascension recta (grados)',
    dec FLOAT COMMENT 'declinacion (grados)',
    u FLOAT COMMENT 'brillo (magnitud) medido en el filtro u',
    g FLOAT COMMENT 'brillo (magnitud) medido en el filtro g',
    r FLOAT COMMENT 'brillo (magnitud) medido en el filtro r',
    i FLOAT COMMENT 'brillo (magnitud) medido en el filtro i',
    z FLOAT COMMENT 'brillo (magnitud) medido en el filtro z',
    field INT COMMENT 'Numero que identifica el area de cielo en la que se encuentra el objeto')
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION '/user/hive/galaxias/SDSS_PhotObj'
TBLPROPERTIES ("skip.header.line.count"="1");
~~~

---

En primer lugar vemos que la tabla se define con el comando "CREATE EXTERNAL TABLE" y a continuación aparece el nombre de la tabla en minúsculas con el sufijo "\_csv" para indicarnos que se trata de datos csv. A continuación y entre paréntesis aparece la lista de columnas con sus tipos y comentarios. Finalmente se especifica el formato con el que hemos guardado los datos (csv separados por comas) y la ubicación de los mismos.

Podemos construir la tabla para el fichero *SDSS_SpecObj.csv* con la siguiente sentencia:

---

~~~
CREATE EXTERNAL TABLE sdss_specobj_csv (
    objid BIGINT COMMENT 'identificador unico del objeto en el catalogo deSDSS',    
    redshift FLOAT COMMENT 'estimacion de corrimiento al rojo, como indicador de la distancia del objeto, se suele utilizar la letra "z" como alias',    
    plate INT COMMENT 'identificador de la instrumentacion del telescopio que se ha utilizado para hacer la medida que ha llevado a la estimacion de redshift',    
    mjd INT COMMENT 'identificador de la fecha en la que se ha tomado la medida (viene del ingles modified Julian date)',    
    fiberid INT COMMENT 'identificador de la instrumentacion del telescopio que se ha utilizado para hacer la medida que ha llevado a la estimacion de redshift',    
    class STRING COMMENT 'nombre de la clase del objeto segun sus caracteristicas espectrales')
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION '/user/hive/galaxias/SDSS_SpecObj'
TBLPROPERTIES ("skip.header.line.count"="1");
~~~

---

Y para comprobar que las tablas han sido creadas, fuera de la consola de HIVE ejecutamos:

---

`
[container]$ hdfs dfs -ls /user/hive/warehouse/galaxias.db/sdss_photobj_csv
Found 1 items
-rw-r--r--   1 impala supergroup  2928399 /user/hive/warehouse/galaxias.db/sdss_photobj_csv/SDSS_PhotObj.csv
[container]$ hdfs dfs -ls /user/hive/warehouse/galaxias.db/sdss_specobj_csv
Found 1 items
-rw-r--r--   1 impala supergroup  1248036 /user/hive/warehouse/galaxias.db/sdss_specobj_csv/SDSS_SpecObj.csv
`

---

<div id='hive-orc' />

#### HIVE: Optimized Row Columnar

Hasta ahora hemos definido el concepto de tabla externa en HIVE como una tabla que relaciona campos de la propia tabla con los campos de un fichero. Y hemos comprobado que podemos acceder a los datos del fichero a través del intérprete HIVE. Sin embargo podemos definir otros tipos de tablas en HIVE. La diferencia entre las tablas externas y las tablas "normales" es que si ejecutamos el comando "DROP TABLE" y se trata de una tabla externa, el fichero a los que apunta la tabla **no** se borrarán. Por el contrario si borramos una tabla "normal", los datos que se encuentran dentro de la tabla **sí** que se eliminarán. Creamos tablas "normales" porque suelen ser mucho más rápidas para realizar análisis sobre los datos en comparación con las tablas externas.

A continuación mostramos la sentencia para crear este nuevo tipo de tablas para el conjunto de datos *SDSS_PhotObj.csv* y *SDSS_SpecObj.csv*:

---

~~~
CREATE TABLE sdss_photobj (
    objid BIGINT COMMENT 'identificador unico del objeto en el catalogo de SDSS',
    fileid BIGINT COMMENT 'alias del parametro \"dr7objid\", identificador que usamos en este proyecto para el fichero que contiene la imagen correspondiente del objeto',
    ra FLOAT COMMENT 'ascension recta (grados)',
    dec FLOAT COMMENT 'declinacion (grados)',
    u FLOAT COMMENT 'brillo (magnitud) medido en el filtro u',
    g FLOAT COMMENT 'brillo (magnitud) medido en el filtro g',
    r FLOAT COMMENT 'brillo (magnitud) medido en el filtro r',
    i FLOAT COMMENT 'brillo (magnitud) medido en el filtro i',
    z FLOAT COMMENT 'brillo (magnitud) medido en el filtro z',
    field INT COMMENT 'Numero que identifica el area de cielo en la que se encuentra el objeto')
STORED AS ORC;
~~~

---

~~~
CREATE TABLE sdss_specobj (
    objid BIGINT COMMENT 'identificador unico del objeto en el catalogo deSDSS',    
    redshift FLOAT COMMENT 'estimacion de corrimiento al rojo, como indicador de la distancia del objeto, se suele utilizar la letra "z" como alias',    
    plate INT COMMENT 'identificador de la instrumentacion del telescopio que se ha utilizado para hacer la medida que ha llevado a la estimacion de redshift',    
    mjd INT COMMENT 'identificador de la fecha en la que se ha tomado la medida (viene del ingles modified Julian date)',    
    fiberid INT COMMENT 'identificador de la instrumentacion del telescopio que se ha utilizado para hacer la medida que ha llevado a la estimacion de redshift',    
    class STRING COMMENT 'nombre de la clase del objeto segun sus caracteristicas espectrales')
STORED AS ORC;
~~~

---

Las diferencias fundamentales entre la definición de la tabla externa y ésta es que no aparece el comando "EXTERNAL", y que el formato con el que la guardamos en este caso es el ORC (Optimized Row Columnar). El formato ORC es un formato eficiente para guardar datos en HIVE, y está diseñado para superar distintas limitaciones de otros formatos, además de mejorar la eficiencia en la lectura, escritura y análisis de los datos. Hay que tener en cuenta que el comando únicamente define la tabla pero que ésta todavía está vacía. Insertaremos los datos en la tabla en las próximas secciones.

<div id='hue' />

### Administración con HUE

Hue es una herramienta con interfaz de usuario web para la gestión de HADOOP, además de una plataforma para construir aplicaciones a medida sobre esta librería UI. Podemos usar esta herramienta para crear las tablas en HIVE y hacer consultas sobre ellas. Veamos como hacerlo.

El puerto desde el que accedemos a HUE varía en función del mapeo que hayamos hecho al arrancar nuestro contenedor, pero por defecto se encuentra en el puerto 8888. En nuestro caso nos dirigimos desde nuestro navegador a la dirección http://localhost:8888.

---

<img src="../images/hue_1.png">

---

Navegamos hasta nuestra base de datos 'galaxias' y creamos la tabla 'sdss_photobj_csv':

---

<img src="../images/hue_2.png">


---

Seleccionamos la estructura y localización en HDFS de nuestro fichero de datos

---

<img src="../images/hue_3.png">

<img src="../images/hue_4.png">

---

Para terminar repetimos el mismo proceso para el conjunto *SDSS_SpecObj.csv*.

Previamente habíamos seleccionado el usuario 'impala' para transferir los archivos a HDFS. Debemos trabajar con el usuario adecuado para no estrar en conflictos de propietarios de archivos. La manupilación de los datos en HDFS mediante HUE son realizadas por el usuario 'impala'. En la siguiente imagen vemos un error de creación de tabla debido a que los datos se encuentran en el directorio `/user/cloudera` cuyo propietario es el usuario 'cloudera'.

---

<img src="../images/hue_5_error.png">

---

Si la creación de tablas ha funcionado deberíamos ver el siguiente mensaje:

---

<img src="../images/hue_6.png">

---

<div id='hue-queries' />

#### HUE Queries

Resulta muy intuitivo trabajar en HUE. Para ver cómo funciona podemos acceder a los datos de la tabla haciendo algunos ejemplos sencillos. Seleccionamos la pestaña 'Query' y elegimos nuestro editor preferido para hacer consultas (en nuestro caso IMPALA):

---

<img src="../images/hue_7.png">

<img src="../images/hue_8.png">

---

Para hacer comprobaciones rápidas es muy común el uso del comando LIMIT tal como muestra la sentencia de la línea 2, para que en caso de que la respuesta contenga muchas entradas, ésta no sea mostrada por pantalla.

En las líneas 3 y 4 se consultan las observaciones para las cuales las variables de brillo 'u' y 'g' registran el valor -9999. En astronomía se usan estos valores para señalar que las medidas no se hicieron correctamente.

En la imagen anterior, del conjunto de consultas se ha ejecutado la correspondiente a la línea 7, la cual junto con la 8 nos muestra el número de diferentes observaciones para la variable 'class'. Las líneas 5 y 6 nos mostrarían las correspondientes tablas para cada clase de observación.

Las filas 9 y 10 nos muestra el número de observaciones. Con esta búsqueda constatamos que cada registro es único (no hay elementos duplicados ya que 'objid' es la clave principal de ambas tablas) y que el número de observaciones entre los distintos datasets concuerdan, siendo un total de 25100 para cada uno.

<div id='hue-orc' />

#### HUE: Optimized Row Columnar

Crear las tablas ORC también se puede hacer de manera interactiva. Tan solo tendremos que rellenar los campos manualmente como se muestra en la siguiente imagen.

---

<img src="../images/hue_9.png">

---

Y repetimos el proceso para el conjunto *sdss_photobj*. Hay que recordar que estas tablas permanecen vacías hasta la inserción de datos.

Para finalizar, gracias a HUE podemos guardar nuestras consultas, ver estadísticas de los datos, preparar dashboards y programar workflows de manipulación de datos entre otras muchas cosas.

<div id='hue-filtrado' />

#### HUE: Filtrado de datos

Vamos a aprovechar que convertimos los datos al formato ORC para deshacernos de aquellos que no vamos a utilizar en nuestro análisis.

Sabemos que nuestro objetivo final es el de clasificar galaxias entre espirales e irregulares, así que no queremos tener estrellas de por medio. Con el siguiente comando, además de rellenar la tabla que acabamos de crear, estamos eliminando todas las entradas clasificadas como "STAR" (se puede ejecutar desde la consola de HIVE).

---

`INSERT INTO TABLE sdss_specobj SELECT * FROM sdss_specobj_csv WHERE class!='STAR';`

---

<img src="../images/hue_10.png">

---

Vamos a eliminar las entradas con valores problemáticos, -9999, que como ya hemos comentado identifican flujo o magnitud incorrectos o extraños. Este sería el comando con el que cargamos la tabla pero únicamente con los valores que nos interesan.

---

`
INSERT INTO TABLE sdss_photobj SELECT * FROM sdss_photobj_csv
WHERE u != -9999.0 
AND g != -9999.0  
AND r != -9999.0  
AND i != -9999.0  
AND z != -9999.0;
`

---

<img src="../images/hue_11.png">

---

<div id='hive-spark' />

### Analítica HIVE - SPARK

Podemos llevar a cabo un análisis exploratorio sobre los datos guardados en HIVE utilizando `pyspark`. A continuación replicamos algunos ejemplos del apartado anterior:

In [1]:
from pyspark.sql import HiveContext

hive = HiveContext(sc)

photobj_csv = hive.table('galaxias.sdss_photobj_csv')
photobj = hive.table('galaxias.sdss_photobj')

specobj_csv = hive.table('galaxias.sdss_specobj_csv')
specobj = hive.table('galaxias.sdss_specobj')

In [2]:
print("photobj data: ", photobj)
print("specobj data: ", specobj)

photobj data:  DataFrame[objid: bigint, fileid: bigint, ra: float, dec: float, u: float, g: float, r: float, i: float, z: float, field: int]
specobj data:  DataFrame[objid: bigint, redshift: float, plate: int, mjd: int, fiberid: int, class: string]


In [3]:
photobj.dtypes

[('objid', 'bigint'),
 ('fileid', 'bigint'),
 ('ra', 'float'),
 ('dec', 'float'),
 ('u', 'float'),
 ('g', 'float'),
 ('r', 'float'),
 ('i', 'float'),
 ('z', 'float'),
 ('field', 'int')]

In [4]:
specobj.dtypes

[('objid', 'bigint'),
 ('redshift', 'float'),
 ('plate', 'int'),
 ('mjd', 'int'),
 ('fiberid', 'int'),
 ('class', 'string')]

In [5]:
photobj.show(3)

+-------------------+------------------+---------+----------+---------+---------+---------+---------+---------+-----+
|              objid|            fileid|       ra|       dec|        u|        g|        r|        i|        z|field|
+-------------------+------------------+---------+----------+---------+---------+---------+---------+---------+-----+
|               null|              null|     null|      null|     null|     null|     null|     null|     null| null|
|1237662224594305220|588017978346111168|156.06728|  36.04212|20.106056|18.252907|17.334618|16.946459|16.623198|  221|
|1237655691937251730|587729970712347008| 215.3721|-2.4439995|20.331465| 18.68993| 17.79132|17.342367|16.958593|   41|
+-------------------+------------------+---------+----------+---------+---------+---------+---------+---------+-----+
only showing top 3 rows



In [6]:
specobj.show(3)

+-------------------+--------+-----+-----+-------+------+
|              objid|redshift|plate|  mjd|fiberid| class|
+-------------------+--------+-----+-----+-------+------+
|               null|    null| null| null|   null|  null|
|1237662224594305220|0.088759| 1957|53415|    148|GALAXY|
|1237655691937251730|0.127493|  917|52400|    111|GALAXY|
+-------------------+--------+-----+-----+-------+------+
only showing top 3 rows



In [7]:
type(specobj), type(photobj)

(pyspark.sql.dataframe.DataFrame, pyspark.sql.dataframe.DataFrame)

In [8]:
hive.sql("SHOW TABLES").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+



In [9]:
hive.registerDataFrameAsTable(photobj_csv, "photobj_csv")
hive.registerDataFrameAsTable(photobj, "photobj")
hive.registerDataFrameAsTable(specobj_csv, "specobj_csv")
hive.registerDataFrameAsTable(specobj, "specobj")

In [10]:
hive.sql("SHOW TABLES").show()

+--------+-----------+-----------+
|database|  tableName|isTemporary|
+--------+-----------+-----------+
|        |    photobj|       true|
|        |photobj_csv|       true|
|        |    specobj|       true|
|        |specobj_csv|       true|
+--------+-----------+-----------+



In [11]:
hive.sql("SELECT * FROM photobj LIMIT 3").show()

+-------------------+------------------+---------+----------+---------+---------+---------+---------+---------+-----+
|              objid|            fileid|       ra|       dec|        u|        g|        r|        i|        z|field|
+-------------------+------------------+---------+----------+---------+---------+---------+---------+---------+-----+
|               null|              null|     null|      null|     null|     null|     null|     null|     null| null|
|1237662224594305220|588017978346111168|156.06728|  36.04212|20.106056|18.252907|17.334618|16.946459|16.623198|  221|
|1237655691937251730|587729970712347008| 215.3721|-2.4439995|20.331465| 18.68993| 17.79132|17.342367|16.958593|   41|
+-------------------+------------------+---------+----------+---------+---------+---------+---------+---------+-----+



In [12]:
hive.sql("SELECT * FROM specobj LIMIT 3").show()

+-------------------+--------+-----+-----+-------+------+
|              objid|redshift|plate|  mjd|fiberid| class|
+-------------------+--------+-----+-----+-------+------+
|               null|    null| null| null|   null|  null|
|1237662224594305220|0.088759| 1957|53415|    148|GALAXY|
|1237655691937251730|0.127493|  917|52400|    111|GALAXY|
+-------------------+--------+-----+-----+-------+------+



In [13]:
hive.sql("SELECT COUNT(*) FROM specobj").show()

+--------+
|count(1)|
+--------+
|   25081|
+--------+



Para eliminar la fila de valores nulos:

In [14]:
hive.sql("SELECT COUNT(*) FROM specobj WHERE objid is not null").show()

+--------+
|count(1)|
+--------+
|   25080|
+--------+



Y registramos dos tablas nuevas sin valores nulos:

In [15]:
photobj_csv = hive.sql("SELECT * FROM photobj_csv WHERE objid is not null")
photobj = hive.sql("SELECT * FROM photobj WHERE objid is not null")
specobj_csv = hive.sql("SELECT * FROM specobj_csv WHERE objid is not null")
specobj = hive.sql("SELECT * FROM specobj WHERE objid is not null")

hive.registerDataFrameAsTable(photobj_csv, "photobj_csv")
hive.registerDataFrameAsTable(photobj, "photobj")
hive.registerDataFrameAsTable(specobj_csv, "specobj_csv")
hive.registerDataFrameAsTable(specobj, "specobj")

In [16]:
hive.sql("SELECT * FROM specobj WHERE objid is null").show()

+-----+--------+-----+---+-------+-----+
|objid|redshift|plate|mjd|fiberid|class|
+-----+--------+-----+---+-------+-----+
+-----+--------+-----+---+-------+-----+



In [17]:
hive.sql("SELECT COUNT(DISTINCT objid) FROM specobj_csv").show()

+---------------------+
|count(DISTINCT objid)|
+---------------------+
|                25054|
+---------------------+



Por construcción nuestra tabla `specobj` no contiene datos de estrellas:

In [18]:
hive.sql("SELECT `class`, COUNT(*) FROM specobj_csv GROUP BY `class`").show()

+------+--------+
| class|count(1)|
+------+--------+
|GALAXY|   25080|
|  STAR|      20|
+------+--------+



In [19]:
hive.sql("SELECT `class`, COUNT(*) FROM specobj GROUP BY `class`").show()

+------+--------+
| class|count(1)|
+------+--------+
|GALAXY|   25080|
+------+--------+



Por construcción nuestra tabla `photobj` tiene 25060 filas:

In [20]:
hive.sql("SELECT COUNT(*) FROM photobj_csv WHERE u != -9999.0 AND g != -9999.0 AND r != -9999.0 AND i != -9999.0 AND z != -9999.0 LIMIT 3").show()

+--------+
|count(1)|
+--------+
|   25060|
+--------+



In [21]:
hive.sql("SELECT COUNT(*) FROM photobj LIMIT 3").show()

+--------+
|count(1)|
+--------+
|   25060|
+--------+

